In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import re
import ast
import patsy
import datetime
import math

from pathlib import Path
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from plotnine import *
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse
from collections import defaultdict
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from plotnine.data import mpg
from mizani.formatters import percent_format



In [4]:
path = Path(os.getcwd())
tech_prep = os.path.join(str(path), "utils")
sys.path.append(tech_prep)
from py_helper_functions import *

In [6]:
main_df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/DA3/main/Assignment%202/lisbon_listings.csv', delimiter=",",encoding="utf-8")
amenities_cleaned_df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/DA3/main/Assignment%202/unique_amenities_cleaned.csv', delimiter=",",encoding="utf-8")


In [7]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22605 entries, 0 to 22604
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            22605 non-null  int64  
 1   listing_url                                   22605 non-null  object 
 2   scrape_id                                     22605 non-null  int64  
 3   last_scraped                                  22605 non-null  object 
 4   source                                        22605 non-null  object 
 5   name                                          22605 non-null  object 
 6   description                                   22586 non-null  object 
 7   neighborhood_overview                         13429 non-null  object 
 8   picture_url                                   22605 non-null  object 
 9   host_id                                       22605 non-null 

In [8]:
main_df = main_df[main_df["property_type"].str.contains(r'^Entire', regex=True)] # fitlering for entire property types
main_df = main_df.loc[(main_df['accommodates'] >= 2) & (main_df['accommodates'] <= 6)] # fitlering for 2-6 accommodates

In [10]:
pd.set_option('display.max_rows',None)

In [11]:
main_df.head().T

,0,1,2,3,5
id,6499,25659,119120,29248,29396
listing_url,https://www.airbnb.com/rooms/6499,https://www.airbnb.com/rooms/25659,https://www.airbnb.com/rooms/119120,https://www.airbnb.com/rooms/29248,https://www.airbnb.com/rooms/29396
scrape_id,20230911050651,20230911050651,20230911050651,20230911050651,20230911050651
last_scraped,2023-09-11,2023-09-12,2023-09-12,2023-09-12,2023-09-12
source,city scrape,city scrape,city scrape,city scrape,city scrape
name,Rental unit in Lisbon · ★4.52 · 1 bedroom · 3 ...,Rental unit in Lisboa · ★4.81 · 1 bedroom · 1 ...,Rental unit in Lisbon · ★4.89 · 2 bedrooms · 2...,Rental unit in Lisbon · ★4.57 · 1 bedroom · 2 ...,Rental unit in Lisbon · ★4.79 · 1 bedroom · 1 ...
description,"This apartment is all about Location, next to ...","Most traditional, charming neighborhood of Lis...",Charming two bedroom with City and River Views...,<b>The space</b><br />Located in the city cent...,Feel at home in the historic centre of Lisbon....
neighborhood_overview,"To get to the city center, tram 15 stops in f...",Alfama is the oldest and most picturesque quar...,Lapa is a unique neighborhood with façades cov...,NaN,Great neighborhood in the old part of town. Am...
picture_url,https://a0.muscache.com/pictures/6422ee92-c84e...,https://a0.muscache.com/pictures/miso/Hosting-...,https://a0.muscache.com/pictures/1012151/4bc3c...,https://a0.muscache.com/pictures/4212cc65-2046...,https://a0.muscache.com/pictures/163913/7d622c...
host_id,14455,107347,387186,125768,126415


In [12]:
main_df.shape

(14347, 75)

In [16]:
main_df['amenities'][:15]

0     ["Shampoo", "River view", "Dishes and silverwa...
1     ["Smoke alarm", "Shampoo", "Dishes and silverw...
2     ["Smoke alarm", "Shampoo", "River view", "Dish...
3     ["Dishes and silverware", "River view", "Refri...
5     ["Dishes and silverware", "Books and reading m...
6     ["Dishes and silverware", "Refrigerator", "Sto...
9     ["Smoke alarm", "Dishes and silverware", "Book...
13    ["Cleaning available during stay", "Shampoo", ...
14    ["Cooking basics", "Dishes and silverware", "W...
15    ["Dishes and silverware", "Refrigerator", "Cri...
16    ["Dishes and silverware", "Refrigerator", "Sto...
18    ["Smoke alarm", "Free dryer \u2013 In unit", "...
20    ["Indoor fireplace", "Dryer", "Washer", "Wifi"...
21    ["Smoke alarm", "Shampoo", "Refrigerator", "Cr...
22    ["Hair dryer", "Wifi", "Kitchen", "Iron", "TV ...
Name: amenities, dtype: object

In [18]:
main_df['bathrooms'].value_counts()

Series([], Name: count, dtype: int64)

In [19]:
main_df['bathrooms_text'].value_counts()

bathrooms_text
1 bath       10570
2 baths       2468
1.5 baths      650
2.5 baths      303
3 baths        262
3.5 baths       40
4 baths         21
0 baths         19
4.5 baths        6
5 baths          2
6 baths          1
Half-bath        1
5.5 baths        1
Name: count, dtype: int64

In [20]:
main_df['bedrooms'].value_counts()

bedrooms
1.0     6410
2.0     5496
3.0     1585
4.0       95
5.0        8
6.0        6
10.0       1
Name: count, dtype: int64

In [ ]:
main_df['bathrooms'].value_counts()

In [21]:
main_df['price'].describe()

count       14347
unique        493
top       $100.00
freq          396
Name: price, dtype: object